In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
#from exps_poisson_ball import setup_poisson_balls, run_exp_poisson_balls
from exps_textured_ball import setup_textured_balls, run_exp_textured_balls

rpm_variants = ['VAE', 'VI', 'amortized']  #, 'termporal']
amortize_ivis = ['none', 'full', 'use_q']

seeds = [2]

Ns = [10]
batch_sizes = [8]
epochses = [500]

conf_exp = {
    'T' : 50,
    'K' : 1,
    'J' : 10,
    'init_rb_bandwidth' : 1000.0,
    #'init_diag_val' : 0.4,
    #'init_off_val' : -1.0,
    'lr' : 1e-3,
    'store_results' : 1,
    'scale_th' : 0.15,
    'shape_max_0' : 1000,
    'ball_sigma2' : 0.01
}

temporal = 1.
iviNatParametrization = 'delta'
normalized_factors = False

for i in range(len(Ns)):
    N, batch_size, epochs = Ns[i], batch_sizes[i], epochses[i]
    for seed in seeds:
        for rpm_variant in rpm_variants:
            for amortize_ivi in amortize_ivis:
                run_exp_textured_balls(N=N, model_seed=seed, data_seed=seed,
                                       rpm_variant=rpm_variant,temporal=temporal,amortize_ivi=amortize_ivi,
                                       epochs=epochs,batch_size=batch_size, 
                                       optim_init_q=True, optim_init_ivi=True, optim_vae_params=False,
                                       iviNatParametrization=iviNatParametrization,
                                       normalized_factors=normalized_factors,
                                       **conf_exp)




running exp VAE_temp_none_textured_N_10_seed_2 from directory .


(N,T,K,J,batch_size) = (10, 50, 1, 10, 8)
initial prior RBF kernel bandwidth:1000.0
-  initializing model with *non*-normalized factors f(Z|xj) !
pre-training q(Z|X) to match \sum_j etaj(xj) + (1-J)eta0


fitting model.
epoch #0/500, loss=0.732718825340271
epoch #50/500, loss=0.6612141132354736
epoch #100/500, loss=0.3623542785644531
epoch #150/500, loss=0.06098340079188347
epoch #200/500, loss=0.02180153876543045
epoch #250/500, loss=0.01788211241364479
epoch #300/500, loss=0.012884342111647129
epoch #350/500, loss=0.011097465641796589
epoch #400/500, loss=0.009344897232949734
epoch #450/500, loss=0.007478659972548485
done fitting.
choosing initial values for ivi parameters to match q


fitting model.
epoch #0/500, train loss=34.69816970825195, test loss=nan
epoch #50/500, train loss=25.343509674072266, test loss=nan
epoch #100/500, train loss=11.54823112487793, test loss=nan
epoch #150/500, train loss=0.3089828491210

epoch #50/500, train loss=29.576946258544922, test loss=nan
epoch #100/500, train loss=16.07175064086914, test loss=nan
epoch #150/500, train loss=12.268681526184082, test loss=nan
epoch #200/500, train loss=11.161633491516113, test loss=nan
epoch #250/500, train loss=9.792490005493164, test loss=nan
epoch #300/500, train loss=8.814284324645996, test loss=nan
epoch #350/500, train loss=6.97736930847168, test loss=nan
epoch #400/500, train loss=4.820406913757324, test loss=nan
epoch #450/500, train loss=2.8756580352783203, test loss=nan
done fitting.


saving results in directory .\fits\VI_temp_full_textured_N_10_seed_2\VI_temp_full_textured_N_10_seed_2
current git commit: 36f398795fc979ba340ba576fe14e7d6c176a393
done saving results.


running exp VI_temp_use_q_textured_N_10_seed_2 from directory .


(N,T,K,J,batch_size) = (10, 50, 1, 10, 8)
initial prior RBF kernel bandwidth:1000.0
-  initializing model with *non*-normalized factors f(Z|xj) !
pre-training q(Z|X) to match \sum_j etaj(xj

In [13]:
import numpy as np

tmp = np.ones((8, 50, 2))
tmp[:,:,0] = 0.
np.max(np.abs(tmp - (tmp.reshape(-1,2)).reshape(8,50,2)))

0.0

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from utils_data_external import linear_regression_1D_latent as regLatent
from utils_data_external import plot_poisson_balls
import matplotlib.pyplot as plt

#from exps_poisson_ball import init_gaussian_rpm
from exps_textured_ball import init_gaussian_rpm


from rpm import RPMEmpiricalMarginals

N = 100
rpm_variant = 'amortized'
amortize_ivi = 'use_q'
model_seed = 0
temporal = True

if temporal:
    identifier = rpm_variant + '_temp_' + amortize_ivi
else:
    identifier = rpm_variant + '_' + amortize_ivi        
identifier = identifier + '_textured_N_' + str(N) + '_seed_' + str(model_seed)

root = os.curdir
res_dir = 'fits'
fn_base = os.path.join(res_dir, identifier, identifier)

data = torch.tensor(np.load(fn_base + '_train_data.npy'))
true_latent_ext = torch.tensor(np.load(fn_base + '_train_latents.npy'))

#data = torch.tensor(np.load(fn_base + '_test_data.npy'))
#true_latent_ext = torch.tensor(np.load(fn_base + '_test_latents.npy'))


exp_dict = np.load(fn_base + '_exp_dict.npz', allow_pickle=True)['arr_0'].tolist()
N,J,K,T = exp_dict['N'],exp_dict['J'],exp_dict['K'],exp_dict['T']
#init_diag_val = exp_dict['init_diag_val']
#init_off_val = exp_dict['init_off_val']
init_rb_bandwidth = exp_dict['init_rb_bandwidth']

ls_train = np.load(fn_base + '_loss_train.npy')
ls_test = np.load(fn_base + '_loss_test.npy')

xjs = [data[:,j] for j in range(J)]
pxjs = RPMEmpiricalMarginals(xjs)
observations = (torch.stack(xjs, dim=-1),)


obs_locs = torch.linspace(0,1,T).reshape(-1,1)
model = init_gaussian_rpm(N, J, K, T, pxjs,
                        init_rb_bandwidth, obs_locs,
                        rpm_variant, temporal, amortize_ivi,
                        epochs=0, batch_size=N
                       )[0]
model.load_state_dict(torch.load(fn_base + '_rpm_state_dict'))


prior = model.joint_model[1]
eta_0 = prior.nat_param
if rpm_variant == 'amortized':    
    eta_q, _ = model.comp_eta_q(xjs, eta_0)
else: 
    eta_q = model.comp_eta_q(xjs, eta_0=eta_0, idx_data=np.arange(N))
EqtZ = prior.log_partition.nat2meanparam(eta_q)

mu = EqtZ[:,:T]
sig2 = torch.diagonal(EqtZ[:,T:].reshape(-1,T,T),dim1=-2,dim2=-1) - mu**2

latent_true, latent_mean_fit, latent_variance_fit, R2 = regLatent(
    latent_true = true_latent_ext,
    latent_mean_fit = mu.unsqueeze(-1), 
    latent_variance_fit = sig2)

epochs = exp_dict['epochs']
plt.plot(np.linspace(1, epochs, len(ls_train)), ls_train,label='train')
plt.plot(np.arange(epochs)+1, ls_test,label='test')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

plot_poisson_balls(observations, 
                   obs_locs=obs_locs.squeeze(-1), 
                   latent_mean_fit=latent_mean_fit.squeeze(-1), 
                   latent_variance_fit=latent_variance_fit)
model.joint_model[1].param_.gamma

In [ ]:
from utils_data_external import sample_textured_balls, plot_poisson_balls
from rpm import RPMEmpiricalMarginals
import numpy as np
import torch

data_seed = 0
N = 10
J = 10
T = 50

np.random.seed(data_seed)
torch.manual_seed(data_seed)

scale_th = 0.15
shape_max_0 = 1000
sigma2 = 0.01

observations_all, true_latent_all, obs_locs = sample_textured_balls(
    num_observation=2*N, dim_observation=J, len_observation=T, 
    scale_th=scale_th, sigma2=sigma2, shape_max_0=shape_max_0, F=10, omega=0.5)

xjs = [observations_all[0][:N,...,j] for j in range(J)]
xjs_test = [observations_all[0][N:,...,j] for j in range(J)] 
true_latent_ext = true_latent_all[:N,:]
true_latent_ext_test = true_latent_all[N:,:]

from rpm import RPMEmpiricalMarginals
pxjs = RPMEmpiricalMarginals(xjs)

plot_poisson_balls(observations_all, 
                   obs_locs=obs_locs.squeeze(-1), 
                   latent_mean_fit=true_latent_all.squeeze(-1), 
                   latent_variance_fit=torch.ones_like(true_latent_all))


# construction site